### 4.1.1 Parse the fasta file (1 point)

How many sequences are contained in the file protein_N_data.fasta?
List the names of the sequences.

In [ ]:
from Bio import SeqIO
from pathlib import Path

# pwd = Path.absolute()

records = SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta")

#Names:
print(f" Names of sequences : {[r.name for r in records]} ")

records = SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta")

#Lengths:
print(f" Length of induvidual sequences : {[len(r.seq) for r in records]} ")

records = SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta")

#Amount of sequences:

records = SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta")

length = len([r for r in records])
print(f" Length of sequences: {length}")



### 4.1.2 Find protein N in a specific coronavirus genome (1 point)

From the sequences in protein_N_data.fasta, find the sequence for which the first letter
in its name is closest in the alphabet to the first letter in your first name. If there
are multiple sequences starting with the same letter, pick one arbitrarily. For the selected
sequence:

**Human_NL63_alpha**

• Find the assembly accession ID in the table above.
• Go to the NCBI website (cf. links above) and find the corresponding genome assembly.
• What are the genomic coordinates (start and end position) of gene N in this genome?
(Hint: follow the RefSeq link)

Link to the gene ["gene"]("https://www.ncbi.nlm.nih.gov/gene/2943504/")

The genes range is : ***26133 - 27266*** 

### 4.1.3 Multiple sequence alignment (1 point)

Build a multiple sequence alignment for the protein_N_data using a multiple sequence
alignment tool of your choice. (Hint: check out the services provided by EMBL’s European
Bioinformatics Institute (EMBL-EBI).)

For this question i used:

https://www.bioinformatics.nl/cgi-bin/emboss/emma

and got out a file, containing aligned sequences.

### 4.1.4 Phylogenetic tree reconstruction (1 point)

Based on the results from the previous step, build a phylogenetic tree. (Hint: at this stage
it is not required to make an “advanced tree”, providing a simple tree is enough). Save the
image of the phylogenetic/guide tree.


Tree was made in this link:

https://www.ebi.ac.uk/Tools/services/web/toolresult.ebi?jobId=simple_phylogeny-I20231002-105856-0957-35098453-p1m

From the MSL

Image:

![tree](outdata2/treeout.png)



### 4.1.5 Interpretation (1 point)

Based on the results from the previous two steps, what do you see? Elaborate with a small
text (3-4 lines): Explain what you observe from the multiple sequence alignment itself (hint:
check the number of conserved sites), and give a short interpretation of the phylogenetic
tree you have constructed.

From the tree i can see that the viruses are grouped together by their "variants". What suprised me is that the tree shows that the variants grouped off quite early. With the exception of gamma and delta, which grouped off from their own common ansector.

Having a look at the alignemnts, one can see some conserved regions across variants, this is very obvious when looking at turkey_gamma and duck_gamma


## 4.2 Step-by-step multiple sequence alignment and phylogenetic tree construction using UPGMA (Total points: 10)

### 4.2.1 Compute pairwise similarities (2 points)

Use the Needleman-Wunsch (dynamic programming) pairwise alignment algorithm to build
a matrix of global alignment scores for each pair of sequences in protein_N_data.fasta.
You can choose between multiple options:
• Implement the Needleman-Wunsch algorithm yourself. (Hint: You have probably done
this already in BINF100)
• Use an existing implementation of the algorithm. (Hint: Check biopython, biojulia)
• Use the needleall command line program from the EMBOSS suite. (Hint: You installed
the whole EMBOSS suite for Assignment 1.)
• Use a webserver such as EMBL-EBI’s EMBOSS Needle service. (Hint: Manually
inputting every pair of sequences will be extremely tedious, though they do provide
APIs.)


In [27]:
from Bio import pairwise2
from Bio import SeqIO
import pandas as pd
from tabulate import tabulate

sequences = list(SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta"))
ids = [record.id for record in sequences]

scores_df = pd.DataFrame(index=ids, columns=ids)

for i in range(len(sequences)):
    for j in range(i, len(sequences)):
        alignments = pairwise2.align.globalxx(sequences[i].seq, sequences[j].seq)

        score = alignments[0].score
        scores_df.iloc[i, j] = score
        scores_df.iloc[j, i] = score


print(tabulate(scores_df, headers='keys', tablefmt='psql'))

+------------------+--------------------+-------------+--------------+---------------+----------------+-------------+---------------+-------------+--------------+-----------------+-------------+---------------+----------------+--------------+---------------+-----------------+----------------+--------------------+-----------------+-----------------+
|                  |   Human_NL63_alpha |   Bat_alpha |   mink_alpha |   camel_alpha |   ferret_alpha |   rat_alpha |   rabbit_beta |   MERS_beta |   HKU24_beta |   England1_beta |   SARS_beta |   whale_gamma |   turkey_gamma |   duck_gamma |   goose_gamma |   sparrow_delta |   wigeon_delta |   nightHeron_delta |   moorhen_delta |   porcine_delta |
|------------------+--------------------+-------------+--------------+---------------+----------------+-------------+---------------+-------------+--------------+-----------------+-------------+---------------+----------------+--------------+---------------+-----------------+----------------+-----

### 4.2.2 Generate a pairwise distance matrix (4 points)
Generate a distance matrix from the score matrix you have created in the previous step.
For this task we will use Feng & Doolittle’s formulation, and we will compute the distance
𝐷 using formula:
𝐷 = − log 𝑆𝑒𝑓𝑓 = − log 𝑆𝑜𝑏𝑠 − 𝑆𝑟𝑎𝑛𝑑
𝑆𝑚𝑎𝑥 − 𝑆𝑟𝑎𝑛𝑑
where
• 𝑆𝑜𝑏𝑠 is the observed pairwise alignment score
• 𝑆𝑚𝑎𝑥 is the best alignment score for both sequences, obtained by taking the average
of the score of aligning either sequence to itself
• 𝑆𝑟𝑎𝑛𝑑 is the expected (average) score for aligning two random sequences of the same
length and residue composition, obtained by random shuffling the nucleotide composition of the two sequences. (Hint: more info about the Feng & Doolittle can be found
at this URL: https://rna.informatik.uni-freiburg.de/Teaching/index.jsp?toolName=
Feng-Doolittle)
Compute 𝑆𝑟𝑎𝑛𝑑 by taking the average score of 10 pairwise alignments between random
sequences with the same sequence compositions as the original sequences

In [29]:
from math import log10
from Bio import pairwise2
from Bio import SeqIO
import pandas as pd
from tabulate import tabulate
import random

def calc_distance(Sobs,Smax,Srand):
    return -log10( (Sobs-Srand) / (Smax - Srand) )

def calc_Srand(seq1,seq2):

    Srand_score = 0

    for _ in range(10):

        random1 = ''.join(random.sample(seq1,len(seq1)))
        random2 = ''.join(random.sample(seq2,len(seq2)))

        alignments = pairwise2.align.globalxx(random1, random2)

        Srand_score += alignments[0].score

    return (Srand_score/10)

sequences = list(SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta"))
ids = [record.id for record in sequences]

distance_df = pd.DataFrame(index=ids, columns=ids)

for i in range(len(sequences)):
    for j in range(i, len(sequences)):
        alignments = pairwise2.align.globalxx(sequences[i].seq, sequences[j].seq)
        alignments_self = pairwise2.align.globalxx(sequences[i].seq, sequences[i].seq)

        Smax = alignments_self[0].score
        Sobs = alignments[0].score
        Srand = calc_Srand(str(sequences[i].seq),str(sequences[j].seq))

        distance = calc_distance(Sobs,Smax,Srand)

        distance_df.iloc[i, j] = distance
        distance_df.iloc[j, i] = distance

print(tabulate(distance_df, headers='keys', tablefmt='psql'))



ValueError: math domain error